<a href="https://colab.research.google.com/github/khurramkhalil/DeepNeuro/blob/master/conv2D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
path =  "/content/drive/My Drive/fNIRS Nature/win6secf/"

In [0]:
import tensorflow
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from collections import Counter
import matplotlib
matplotlib.use('Agg')
import os
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.utils import np_utils
import glob
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.utils import np_utils
global win
win = 60


In [0]:


def reject_outliers(data, m = 2):
    return data[abs(data - np.mean(data)) < m * np.std(data)]

def normalize(data):
    sc = StandardScaler()
    s = sc.fit_transform(data)
    return s

def minmax(data):
    # print(data)
    sc = MinMaxScaler()
    s = sc.fit_transform(data.astype(np.float))
    return s

def rec_plot(s, eps = None, steps = None):
    if eps == None: eps = 1  # (3) distance, if two values within signal are closer than \epsilon then we draw a dot.
    if steps == None: steps = 10 # (4) Embadding time delay
    N = s.size
    S = np.repeat(s[None, :], N, axis = 0)
    Z = np.floor(np.abs(S-S.T) / eps)
    Z[Z > steps] = steps
    return Z

def slidingWindow(sequence, winSize, step = 1):
    """Returns a generator that will iterate through
    the defined chunks of input sequence.  Input sequence
    must be iterable."""
 
    # Verify the inputs
    try: it = iter(sequence)
    except TypeError:
        raise Exception("**ERROR** sequence must be iterable.")
    if not ((type(winSize) == type(0)) and (type(step) == type(0))):
        raise Exception("**ERROR** type(winSize) and type(step) must be int.")
    if step > winSize:
        raise Exception("**ERROR** step must not be larger than winSize.")
    if winSize > len(sequence):
        raise Exception("**ERROR** winSize must not be larger than sequence length.")
 
    # Pre-compute number of chunks to emit
    numOfChunks = int(((len(sequence) - winSize) / step) + 1)
 
    # Do the work
    for i in range(0, numOfChunks * step, step):
        yield sequence[i: i + winSize]


In [0]:
fo = glob.glob("/content/drive/My Drive/fNIRS Nature/o*.csv")
fd = glob.glob("/content/drive/My Drive/fNIRS Nature/d*.csv")

for i in range(len(fo)):
    oxy = pd.read_csv(fo[i])
    doxy = pd.read_csv(fd[i])
    
    oxy = oxy.iloc[:].values
    doxy = doxy.iloc[:].values
    
    
    def makeplt(data): # len(g)
        x = data
        v = 0
        # win = win        # Window size
        step = 30
        noOfSeq = int(((len(oxy[:, 1]) - win) / step) + 1)
        x_train = np.zeros((noOfSeq, win, 36))
        y_train = np.zeros(noOfSeq).astype(np.ndarray)

        chunks = slidingWindow(x, win, step)
        # z = []
        for chunk in chunks:
            ch = chunk[:, :-1]
            lab = chunk[: , -1]
            x_train[v, :, :] = ch
            b = Counter(lab)
            c = b.most_common(1)
            c = list(list(zip(*c))[0])
            c = np.array(c)
            y_train[v] = c[0]
            v += 1
            y_train = y_train.astype('U')
        return x_train[:-1, :], y_train[:-1]

        
    x1, y1 = makeplt(oxy)
    x2, y2 = makeplt(doxy)
    
    '''
    For Hb) + HbR
for i in range(0, 10):
    
    print(i)
    i -= 2
    
    '''
    
    
    # win = win
    ch = 36
    
    
    
    x_train = np.concatenate((x1, x2))
    y_train = np.concatenate((y1, y2))
    
    
    
    X_train = x_train.reshape(x_train.shape[0], 1, win, ch)
    
    from sklearn.preprocessing import LabelEncoder
    
    
    labelencoder_y = LabelEncoder()
    y_train = labelencoder_y.fit_transform(y_train)
    
    Y_train = np_utils.to_categorical(y_train, 4)
    
    X_train, X_test, y_train, y_test = train_test_split(X_train, Y_train, test_size = 0.2)
    
    
    import tensorflow as tf
    from tensorflow.keras.layers import Lambda
    from tensorflow.keras import layers, models, regularizers
    
    
    
    
    
    
    model = models.Sequential()
     
    model.add(layers.Conv2D(32, (3, 3), activation='elu', input_shape = (1, win, ch), data_format = 'channels_first'))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(32, (3, 3), activation='elu', kernel_regularizer = regularizers.l2(0.01)))
    model.add(layers.Dropout(0.1))
    # model.add(layers.Conv2D(64, (3, 3), activation='relu'))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(64, activation='elu', kernel_regularizer = regularizers.l2(0.001)))
    #model.add(Lambda(lambda v: tf.cast(tf.compat.v1.spectral.fft(tf.cast(v,dtype=tf.complex64)),tf.float32)))
    model.add(layers.Dropout(0.2))
    model.add(layers.Dense(64, activation='elu', kernel_regularizer = regularizers.l2(0.001)))
    # model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4, activation='softmax'))
    
    # model.add(Dense(4, activation = 'softmax'))
    
    model.compile(loss = 'categorical_crossentropy',
                  optimizer = 'adam', metrics = ['accuracy'])
    
    hist = model.fit(X_train, y_train, validation_data = (X_test, y_test),
                     epochs = 500, batch_size = 32, shuffle = True, verbose = 0)
    model.save_weights(path + 'secondw60_try_%d.h5'%i)  # always save your weights after training or during training
    
    
    y_pred = model.predict(X_test)
    y_pred = (y_pred > 0.5)
    
    
    dpi = 300
    
    plt.close()
    plt.figure(1)
    plt.plot(hist.history['accuracy'])
    plt.plot(hist.history['val_accuracy'])
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.ylim(0, 1)
    plt.xlabel('Epoch')
    # plt.xlim(0, 200)
    plt.legend(['Train', 'Validation'], loc='lower right')
    # plt.grid()
    plt.savefig(path + "Accuracyw60_figure(Subject)_%d.png"%i, dpi = dpi, bbox_inches='tight')
    
    #plt.show()
    
    # "Loss"
    plt.close()
    plt.figure(2)
    plt.plot(hist.history['loss'])
    plt.plot(hist.history['val_loss'])
    plt.title('Model Loss')
    plt.ylabel('Loss')
    # plt.ylim(0, 1)
    plt.xlabel('Epoch')
    # plt.xlim(0, 200)
    plt.legend(['Train', 'Validation'], loc='upper right')
    # plt.grid()
    plt.savefig(path + "Lossw60_figure(Subject)_%d.png"%i, dpi = dpi, bbox_inches='tight')
    np.save(path + 'myw60_history_%d.npy'%i, hist.history)

#history=np.load('my_history.npy',allow_pickle='TRUE').item()